In [1]:
%matplotlib inline
import requests
from matplotlib.patches import Circle, Rectangle, Arc
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
Atlanta_Hawks_shots_heaves_removed = pd.read_csv('data/Atlanta_Hawks_shots_heaves_removed.csv')

In [3]:
Atlanta_Hawks_shots_heaves_removed

,Unnamed: 0,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,4868,Shot Chart Detail,21500001,138,203145,Kent Bazemore,1610612737,Atlanta Hawks,2,10,9,Missed Shot,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,12,-1,1,0,20151027,ATL,DET
1,4869,Shot Chart Detail,21500001,152,203145,Kent Bazemore,1610612737,Atlanta Hawks,2,8,28,Missed Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-2,2,1,0,20151027,ATL,DET
2,4870,Shot Chart Detail,21500001,260,203145,Kent Bazemore,1610612737,Atlanta Hawks,3,9,54,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-183,174,1,0,20151027,ATL,DET
3,4871,Shot Chart Detail,21500019,7,203145,Kent Bazemore,1610612737,Atlanta Hawks,1,11,23,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-27,0,1,1,20151029,NYK,ATL
4,4872,Shot Chart Detail,21500019,197,203145,Kent Bazemore,1610612737,Atlanta Hawks,2,3,44,Missed Shot,Reverse Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,17,8,1,0,20151029,NYK,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6903,188095,Shot Chart Detail,21501221,352,200794,Paul Millsap,1610612737,Atlanta Hawks,3,8,44,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,26,9,262,1,0,20160413,WAS,ATL
6904,188096,Shot Chart Detail,21501221,397,200794,Paul Millsap,1610612737,Atlanta Hawks,3,4,18,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,14,51,1,0,20160413,WAS,ATL
6905,188097,Shot Chart Detail,21501221,399,200794,Paul Millsap,1610612737,Atlanta Hawks,3,4,16,Missed Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-12,-5,1,0,20160413,WAS,ATL
6906,188098,Shot Chart Detail,21501221,422,200794,Paul Millsap,1610612737,Atlanta Hawks,3,2,36,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,-50,51,1,0,20160413,WAS,ATL


In [4]:
Atlanta_Hawks_shots_heaves_removed[Atlanta_Hawks_shots_heaves_removed.SHOT_ZONE_AREA == 'Back Court(BC)']

,Unnamed: 0,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
1618,58714,Shot Chart Detail,21500878,113,203501,Tim Hardaway Jr.,1610612737,Atlanta Hawks,1,0,0,Made Shot,Running Pull-Up Jump Shot,3PT Field Goal,Above the Break 3,Back Court(BC),Back Court Shot,41,97,401,1,1,20160228,ATL,CHA
6552,187743,Shot Chart Detail,21500808,475,200794,Paul Millsap,1610612737,Atlanta Hawks,4,5,43,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Back Court(BC),Back Court Shot,40,-2,400,1,0,20160219,ATL,MIA


In [5]:
Atlanta_Hawks_shots_heaves_removed.SHOT_DISTANCE.unique()

array([ 1,  0, 25,  2, 23, 13, 24,  5, 12, 27,  9, 20,  3, 19,  7, 22, 16,
        6, 17, 26, 18,  4, 21,  8, 14, 10, 15, 11, 29, 28, 40, 31, 41, 33,
       36, 39, 38, 45])

In [6]:
def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [9]:
@interact
def shot_chart(GameNo = ['All',21500001,
 21500019,
 21500026,
 21500039,
 21500055,
 21500062,
 21500081,
 21500086,
 21500101,
 21500117,
 21500129,
 21500148,
 21500159,
 21500169,
 21500191,
 21500210,
 21500222,
 21500233,
 21500242,
 21500256,
 21500272,
 21500286,
 21500328,
 21500331,
 21500347,
 21500360,
 21500376,
 21500389,
 21500407,
 21500413,
 21500429,
 21500442,
 21500459,
 21500470,
 21500506,
 21500521,
 21500536,
 21500551,
 21500577,
 21500599,
 21500602,
 21500620,
 21500639,
 21500643,
 21500660,
 21500672,
 21500687,
 21500692,
 21500715,
 21500723,
 21500736,
 21500750,
 21500771,
 21500780,
 21500796,
 21500808,
 21500819,
 21500836,
 21500865,
 21500878,
 21500895,
 21500921,
 21500929,
 21500947,
 21500959,
 21500974,
 21500984,
 21501007,
 21501015,
 21501029,
 21501048,
 21501056,
 21501076,
 21501085,
 21501099,
 21501113,
 21501131,
 21501157,
 21501173,
 21501188,
 21501203,
 21501221], Player = ['All', 'Al Horford',
 'Dennis Schroder',
 'Edy Tavares',
 'Jeff Teague',
 'Justin Holiday',
 'Kent Bazemore',
 'Kirk Hinrich',
 'Kris Humphries',
 'Kyle Korver',
 'Lamar Patterson',
 'Mike Muscala',
 'Mike Scott',
 'Paul Millsap',
 'Shelvin Mack',
 'Thabo Sefolosha',
 'Tiago Splitter',
 'Tim Hardaway Jr.'],
              Quarter = ['All', 1, 2, 3, 4, 5, 6],
              MakeOrMiss = ['All','Made Shot', 'Missed Shot'],
              ShotType = ['All','2PT Field Goal', '3PT Field Goal'],
              ShotZone = ['All','Center(C)', 'Left Side Center(LC)', 'Right Side Center(RC)', 'Left Side(L)', 'Right Side(R)','Back Court(BC)'],
              ShotDistance = ['All','Less Than 8 ft.','8-16 ft.','16-24 ft.', '24+ ft.', 'Back Court Shot'],
              Opponent = ['All',
 'BKN',
 'BOS',
 'CHA',
 'CHI',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHX',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS'],
              HmOrAw = ['All', 'Home', 'Away']):
    
    new_df = Atlanta_Hawks_shots_heaves_removed
    
    def game(df):
        if GameNo == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.GAME_ID == GameNo]
        return df
    
    def player(df):
        if Player == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.PLAYER_NAME == Player]
        return df
    
    def quarter(df):
        if Quarter == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.PERIOD == Quarter]
        return df
    
    def makeormiss(df):
        if MakeOrMiss == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.EVENT_TYPE == MakeOrMiss]
        return df
    
    def shottype(df):
        if ShotType == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.SHOT_TYPE == ShotType]
        return df
    
    def shotzone(df):
        if ShotZone == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.SHOT_ZONE_AREA == ShotZone]
        return df
    
    def shotdistance(df):
        if ShotDistance == 'All':
            df = new_df
        else:
            df = new_df.loc[new_df.SHOT_ZONE_RANGE == ShotDistance]
        return df
    
    def opponent(df):
        if Opponent == 'All':
            df = new_df
        else:
            df = new_df.loc[(new_df.HTM == Opponent) | (new_df.VTM == Opponent)]
        return df
    
    def hmoraw(df):
        if HmOrAw == 'All':
            df = new_df
        elif HmOrAw == 'Home':
            df = new_df.loc[new_df.HTM == 'ATL']
        else:
            df = new_df.loc[new_df.VTM == 'ATL']
        return df
            
    new_df = game(new_df)
    new_df = player(new_df)
    new_df = quarter(new_df)
    new_df = makeormiss(new_df)
    new_df = shottype(new_df)
    new_df = shotzone(new_df)
    new_df = shotdistance(new_df)
    new_df = opponent(new_df)
    new_df = hmoraw(new_df)
    

    plt.figure(figsize=(12,11))
    colors = {'Made Shot': '#E03A3E', 'Missed Shot': '#C1D32F'}
    sns.scatterplot(x=new_df.LOC_X, y=new_df.LOC_Y, hue=new_df.EVENT_TYPE, palette=colors, s = 100)
    draw_court()
# Adjust plot limits to just fit in half court
    plt.xlim(-250,250)
# Descending values along th y axis from bottom to top
# in order to place the hoop by the top of plot
    plt.ylim(422.5, -47.5)
# get rid of axis tick labels
    plt.tick_params(labelbottom=False, labelleft=False)
    plt.xlabel('Baseline', fontsize = 20)
    plt.ylabel('Sideline', fontsize = 20)
    plt.legend(loc = 'lower right', fontsize = 15)
    plt.show()

interactive(children=(Dropdown(description='GameNo', options=('All', 21500001, 21500019, 21500026, 21500039, 2…

In [10]:
chart_cols = ['GAME_ID', 'PLAYER_NAME', 'PERIOD', 'EVENT_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE']

In [11]:
games = []
players = []
periods = []
eventtypes = []
shottypes = []
shotzoneareas = []
shot_zone_range = []
opponents = []

def get_games(df, col_list):
    games = list(df[col_list[0]].unique())
    games.append('All')
    games.insert(0, games.pop(-1))
    return games

In [12]:
games = list(Atlanta_Hawks_shots_heaves_removed.GAME_ID.unique())

In [13]:
games.append('All')

In [14]:
games.insert(0, games.pop(-1))

In [15]:
games = get_games(Atlanta_Hawks_shots_heaves_removed, chart_cols)

In [16]:
games

['All',
 21500001,
 21500019,
 21500026,
 21500039,
 21500055,
 21500062,
 21500081,
 21500086,
 21500101,
 21500117,
 21500129,
 21500148,
 21500233,
 21500242,
 21500256,
 21500272,
 21500286,
 21500328,
 21500331,
 21500347,
 21500360,
 21500376,
 21500389,
 21500407,
 21500413,
 21500429,
 21500442,
 21500459,
 21500470,
 21500506,
 21500521,
 21500536,
 21500551,
 21500577,
 21500599,
 21500602,
 21500620,
 21500639,
 21500643,
 21500660,
 21500672,
 21500687,
 21500692,
 21500715,
 21500723,
 21500736,
 21500750,
 21500771,
 21500780,
 21500796,
 21500808,
 21500819,
 21500836,
 21500865,
 21500878,
 21500895,
 21500921,
 21500929,
 21500947,
 21500959,
 21500974,
 21500984,
 21501007,
 21501029,
 21501048,
 21501056,
 21501076,
 21501085,
 21501099,
 21501113,
 21501131,
 21501157,
 21501188,
 21501203,
 21501221,
 21500159,
 21500169,
 21500191,
 21500210,
 21500222,
 21501015,
 21501173]